### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
print(today)

2025-08-28


In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-08-28
Yesterday: 2025-08-27


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-08-27'
ORDER BY a.name



(16, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-08-27' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,AIMIRT,2025-08-22,2025-08-27,9.55,9.70,331200,9.70,9.55,B,SET,0.15,1.57
1,CPF,2025-08-22,2025-08-27,24.30,23.90,129833584,24.50,23.90,B,SET50,-0.40,-1.65
2,GVREIT,2025-08-27,2025-08-27,6.45,6.20,4390028,6.45,6.20,B,SET,-0.25,-3.88
3,JMART,2025-08-26,2025-08-27,8.60,8.35,31361924,8.60,8.35,B,SET50,-0.25,-2.91
4,JMT,2025-08-26,2025-08-27,12.20,11.80,80415480,12.20,11.20,B,SET50,-0.40,-3.28
5,ORI,2025-08-26,2025-08-27,2.28,2.20,79405704,2.36,2.20,B,SET100,-0.08,-3.51
6,PTT,2025-08-26,2025-08-27,32.25,31.75,616557932,32.25,31.75,B,SET50,-0.50,-1.55
7,RCL,2025-08-22,2025-08-27,26.75,28.25,16978064,28.25,26.75,B,SET100,1.50,5.61
8,TFFIF,2025-08-21,2025-08-27,6.05,6.00,24284176,6.10,6.00,B,SET,-0.05,-0.83
9,WHAIR,2025-08-27,2025-08-27,5.55,5.50,1135608,5.55,5.35,B,SET,-0.05,-0.90


In [20]:
df.shape

(16, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,AIMIRT,2025-08-22,2025-08-27,9.55,9.70,"331,200",9.70,9.55,1.57%,B
1,CPF,2025-08-22,2025-08-27,24.30,23.90,"129,833,584",24.50,23.90,-1.65%,B
2,GVREIT,2025-08-27,2025-08-27,6.45,6.20,"4,390,028",6.45,6.20,-3.88%,B
3,JMART,2025-08-26,2025-08-27,8.60,8.35,"31,361,924",8.60,8.35,-2.91%,B
4,JMT,2025-08-26,2025-08-27,12.20,11.80,"80,415,480",12.20,11.20,-3.28%,B
5,ORI,2025-08-26,2025-08-27,2.28,2.20,"79,405,704",2.36,2.20,-3.51%,B
6,PTT,2025-08-26,2025-08-27,32.25,31.75,"616,557,932",32.25,31.75,-1.55%,B
7,RCL,2025-08-22,2025-08-27,26.75,28.25,"16,978,064",28.25,26.75,5.61%,B
8,TFFIF,2025-08-21,2025-08-27,6.05,6.00,"24,284,176",6.10,6.00,-0.83%,B
9,WHAIR,2025-08-27,2025-08-27,5.55,5.50,"1,135,608",5.55,5.35,-0.90%,B


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(16, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(169, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(16, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,AIMIRT,2025-08-22,2025-08-27,9.55,9.70,331200,9.70,9.55,1.57,B,9.70,0.00,28245,2025-08-28
1,CPF,2025-08-22,2025-08-27,24.30,23.90,129833584,24.50,23.90,-1.65,B,23.90,0.00,22555335,2025-08-28
2,GVREIT,2025-08-27,2025-08-27,6.45,6.20,4390028,6.45,6.20,-3.88,B,6.20,0.00,367600,2025-08-28
3,JMART,2025-08-26,2025-08-27,8.60,8.35,31361924,8.60,8.35,-2.91,B,8.45,0.10,8113925,2025-08-28
4,JMT,2025-08-26,2025-08-27,12.20,11.80,80415480,12.20,11.20,-3.28,B,11.90,0.10,5735498,2025-08-28
5,ORI,2025-08-26,2025-08-27,2.28,2.20,79405704,2.36,2.20,-3.51,B,2.20,0.00,4936860,2025-08-28
6,PTT,2025-08-26,2025-08-27,32.25,31.75,616557932,32.25,31.75,-1.55,B,31.50,-0.25,37533807,2025-08-28
7,RCL,2025-08-22,2025-08-27,26.75,28.25,16978064,28.25,26.75,5.61,B,28.00,-0.25,1480753,2025-08-28
8,TFFIF,2025-08-21,2025-08-27,6.05,6.00,24284176,6.10,6.00,-0.83,B,5.95,-0.05,661703,2025-08-28
9,WHAIR,2025-08-27,2025-08-27,5.55,5.50,1135608,5.55,5.35,-0.90,B,5.55,0.05,417336,2025-08-28


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
AIMIRT   2025-08-22 2025-08-28   9.55   9.70   1.57%     331,200   9.70   9.55 B Update
CPF      2025-08-22 2025-08-28  24.30  23.90  -1.65% 129,833,584  24.50  23.90 B Update
DIF      2025-08-26 2025-08-28   8.25   8.20  -0.61%  51,176,552   8.25   8.10 S Update
GVREIT   2025-08-27 2025-08-28   6.45   6.20  -3.88%   4,390,028   6.45   6.20 B Update
IVL      2025-08-27 2025-08-28  23.50  21.90  -6.81% 105,419,488  23.50  21.90 S Update
JMART    2025-08-28 2025-08-28   8.35   8.45    1.2%  31,361,924   8.60   8.35 B Insert
JMT      2025-08-28 2025-08-28  11.80  11.90   0.85%  80,415,480  12.20  11.20 B Insert
MCS      2025-08-28 2025-08-28   8.25   8.30   0.61%   2,084,600   8.65   8.25 S Insert
NER      2025-08-28 2025-08-28   4.26   4.28   0.47%  88,520,356   4.60   4.24 S Insert
ORI      2025-08-26 2025-08-28  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [55]:
sql = """
    SELECT a.name,fm_date,to_date,fm_price,to_price,
    a.qty,a.max_price,a.min_price,t.status,t.market
    FROM sales a 
    JOIN stocks t ON a.name = t.name 
    WHERE to_date = '{}' AND t.status IN ("B","I", "O", "S") 
    ORDER BY t.status, a.name
""".format(today)
print(sql)


    SELECT a.name,fm_date,to_date,fm_price,to_price,
    a.qty,a.max_price,a.min_price,t.status,t.market
    FROM sales a 
    JOIN stocks t ON a.name = t.name 
    WHERE to_date = '2025-08-28' AND t.status IN ("B","I", "O", "S") 
    ORDER BY t.status, a.name



In [57]:
df = pd.read_sql(sql, conlite)
df.shape

(16, 10)

In [59]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,AIMIRT,2025-08-22,2025-08-28,9.55,9.70,"662,400",9.70,9.55,B,SET,0.15,1.57%
1,CPF,2025-08-22,2025-08-28,24.30,23.90,"259,667,168",24.50,23.90,B,SET50,-0.40,-1.65%
2,GVREIT,2025-08-27,2025-08-28,6.45,6.20,"8,780,056",6.45,6.20,B,SET,-0.25,-3.88%
3,JMART,2025-08-28,2025-08-28,8.35,8.45,"31,361,924",8.60,8.35,B,SET50,0.10,1.20%
4,JMT,2025-08-28,2025-08-28,11.80,11.90,"80,415,480",12.20,11.20,B,SET50,0.10,0.85%
5,ORI,2025-08-26,2025-08-28,2.28,2.20,"158,811,408",2.36,2.20,B,SET100,-0.08,-3.51%
6,PTT,2025-08-26,2025-08-28,32.25,31.50,"1,233,115,864",32.25,31.50,B,SET50,-0.75,-2.33%
7,RCL,2025-08-28,2025-08-28,28.25,28.00,"16,978,064",28.25,26.75,B,SET100,-0.25,-0.88%
8,TFFIF,2025-08-21,2025-08-28,6.05,5.95,"48,568,352",6.10,5.95,B,SET,-0.10,-1.65%
9,WHAIR,2025-08-28,2025-08-28,5.50,5.55,"1,135,608",5.55,5.35,B,SET,0.05,0.91%


In [61]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [63]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
12,IVL,2025-08-27,2025-08-28,23.50,21.90,"210,838,976",23.50,21.90,S,SET50,-1.60,-6.81%


In [65]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [67]:
conlite.close()

In [69]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-08-28 21:21:50
